In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
sqlcontext=SQLContext(sc)

In [2]:
examDF=spark.read.csv("dataset.csv",header=True,inferSchema=True)

In [3]:
examDF.show()
examDF.columns

+-------+-----------------------------+---------+
|Student|% achieved in the assignments|Pass exam|
+-------+-----------------------------+---------+
|  Alice|                          70%|      Yes|
|    Bob|                          36%|       No|
|  Carol|                          95%|      Yes|
|    Dan|                          63%|      Yes|
|    Eve|                          43%|       No|
|  Frank|                          84%|      Yes|
|  Grace|                          54%|      Yes|
|  Heidi|                          15%|       No|
|   Ivan|                          21%|       No|
|   Judy|                          91%|      Yes|
|Mallory|                          34%|       No|
|   null|                         null|     null|
+-------+-----------------------------+---------+



['Student', '% achieved in the assignments', 'Pass exam']

In [4]:
examDF.printSchema()

root
 |-- Student: string (nullable = true)
 |-- % achieved in the assignments: string (nullable = true)
 |-- Pass exam: string (nullable = true)



In [5]:
examDF=examDF.withColumnRenamed("% achieved in the assignments","%Achieved")
examDF.columns

['Student', '%Achieved', 'Pass exam']

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
def p2n(x): #Percentage to integer // removing percentage sign
    return int(x.strip('%'))
p2nudf=udf(lambda x: p2n(x),IntegerType())


In [7]:
examDF=examDF.withColumn("%Achieved",p2nudf(examDF["%Achieved"]))

In [8]:
examDF.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Program Files\spark-3.1.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 604, in main
  File "C:\Program Files\spark-3.1.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 596, in process
  File "C:\Program Files\spark-3.1.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\Program Files\spark-3.1.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "C:\Program Files\spark-3.1.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 200, in _batched
    for item in iterator:
  File "C:\Program Files\spark-3.1.1-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 450, in mapper
  File "C:\Program Files\spark-3.1.1-

In [ ]:
examDF.printSchema()

### Calculating mean,std and variance 

In [ ]:
# Mean YES
examDF.filter(examDF["Pass exam"]=="Yes").groupBy("Pass exam").agg({"%Achieved":"mean"}).show()

In [ ]:
# STD Yes
examDF.filter(examDF["Pass exam"]=="Yes").groupBy("Pass exam").agg({"%Achieved":"std"}).show()

In [ ]:
# Variance Yes
examDF.filter(examDF["Pass exam"]=="Yes").groupBy("Pass exam").agg({"%Achieved":"variance"}).show()

In [ ]:
# Mean NO
examDF.filter(examDF["Pass exam"]=="No").groupBy("Pass exam").agg({"%Achieved":"mean"}).show()

In [ ]:
# STD no
examDF.filter(examDF["Pass exam"]=="No").groupBy("Pass exam").agg({"%Achieved":"std"}).show()

In [ ]:
# Variance No
examDF.filter(examDF["Pass exam"]=="No").groupBy("Pass exam").agg({"%Achieved":"variance"}).show()

### Creating a Naive bayes model to predict whether a student with specific marks will pass the exam or not

In [ ]:
examDF.show()
examDF.printSchema()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
str_indexer=StringIndexer().setInputCol("Pass exam").setOutputCol("label")

In [ ]:
indexed_examDF=str_indexer.fit(examDF).transform(examDF)

In [ ]:
indexed_examDF.show()
indexed_examDF.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
va=VectorAssembler().setInputCols(["%Achieved"]).setOutputCol("features")

In [ ]:
training_DF=va.transform(indexed_examDF)

In [ ]:
training_DF.show()

In [ ]:
train,test=training_DF.randomSplit([0.7,0.3],1234)

In [ ]:
from pyspark.ml.linalg import *


In [ ]:
#Applying Naive bayes classification
from pyspark.ml.classification import LogisticRegression, NaiveBayes

In [ ]:
lr=LogisticRegression(featuresCol="features",labelCol="label")

In [ ]:
# training the data
model=lr.fit(training_DF)
#pred=model.transform(test)


In [ ]:
model.predictProbability(DenseVector([49.0]))

In [ ]:
#evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator=MulticlassClassificationEvaluator()

In [ ]:
evaluator.evaluate(pred)

In [ ]:
pred.show()

In [ ]:
from pyspark.ml.linalg import *
model.predictProbability(DenseVector([48.0]))